In [1]:
import pandas as pd
import os

In [2]:
# include the utils directory in the path
import sys
sys.path.append(os.path.abspath(os.path.join('../')))

In [3]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# -------------------- DEFINE CUSTOM TEMPLATE -------------------- #
pio.templates['draft'] = go.layout.Template(layout=dict(
    margin=dict(l=50, r=50, b=50, t=50),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
)
))
pio.templates.default = "plotly_white+draft"

### Fragment Results

In [4]:
# fragment_result = pd.read_csv('./Pos_training/Fragment_all_results copy.csv')
# fragment_result.sort_values(by=['scenario', 'ssl_model', 'head_model'], inplace=True)
# fragment_result.to_csv('./Pos_training/Fragment_all_results copy.csv', index=False)

In [13]:
fragment_result = pd.read_csv('./Pos_training/Fragment_all_results copy.csv')
fragment_result

,ssl_model,head_model,calibration,scenario,dataset,avg_loss,acc,acc_std,f1,f1_std,recall,recall_std,precision,precision_std
0,TSTCC,MLP,linearprobing,ECG,Fragment,0.5509,0.5317,0.0063,0.4072,0.0076,0.5509,0.0080,1.1092,0.0158
1,TSTCC,MLP,linearprobing,All,Fragment,0.5043,0.4837,0.0066,0.3457,0.0084,0.5043,0.0051,1.0121,0.0300
2,TSTCC,FCN,finetuning,All,Fragment,0.2810,0.1680,0.1165,0.1032,0.0564,0.2810,0.1058,1.8616,0.1345
3,TSTCC,FCN,linearprobing,ECG,Fragment,0.1951,0.1054,0.0496,0.0825,0.0337,0.1951,0.0570,1.6465,0.0328
4,TSTCC,FCN,linearprobing,All,Fragment,0.1988,0.0939,0.0192,0.1003,0.0306,0.1988,0.0192,1.6779,0.0777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,TS2Vec,FCN,finetuning,5 & 6 combined,Fragment,0.2098,0.1451,0.1220,0.1360,0.0865,0.2098,0.1119,2.2617,0.8436
104,TSTCC,MLP,linearprobing,5 & 6 combined,Fragment,0.5055,0.4996,0.0030,0.3817,0.0036,0.5055,0.0034,1.2150,0.0215
105,TSTCC,FCN,linearprobing,5 & 6 combined,Fragment,0.2417,0.2060,0.0707,0.1767,0.0654,0.2417,0.0437,1.6114,0.0387
106,TSTCC,FCN,finetuning,5 & 6 combined,Fragment,0.1828,0.0849,0.0398,0.0932,0.0460,0.1828,0.0482,1.8865,0.0582


In [14]:
# temp_df = fragment_result[['ssl_model','head_model', 'scenario', 'calibration', 'f1','f1_std']].copy()
# df_filtered = temp_df.sort_values(by='f1', ascending=False) \
#                      .drop_duplicates(subset=['scenario','ssl_model','head_model'], keep='first') \
#                      .sort_values(by=['scenario','ssl_model','head_model']).copy()

# df_filtered['latex'] = (df_filtered['f1']*100).round(4).astype(str) + ' ($\pm$ ' + (df_filtered['f1_std']*100).round(4).astype(str) + ')'

# df_filtered.drop(columns=['f1','f1_std'], inplace=True) 
# df_filtered
# # df_filtered.to_csv('./Pos_training/Fragment_all_results_latex.csv')

In [20]:
scenario_order = ['ECG', 'All', 'ECG+Fragment', 'All+Fragment', 
                  'ECG extended', 'ECG continued', 'All extended',
                  '5 & 6 combined']

In [21]:
fig = px.bar(
    fragment_result.query("calibration == 'finetuning' & ssl_model == 'TS2Vec' & scenario != 'ECG continued30'"), 
    x='scenario', y='f1', color='head_model', barmode='group',
    labels={'ssl_model': 'SSL Model', 'f1': 'F1-score', 'scenario': 'Strategy'},
    category_orders={'scenario': scenario_order},
)

fig.update_yaxes(range=[0, 1])
fig.update_xaxes(title_text='', tickfont=dict(size=18))
fig.update_layout(height=400, width=1100, font_size=16, 
                  legend_title_text=None,
                  margin=dict(l=40, r=40, b=30, t=50),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.01, xanchor="center", x=0.5)
)
fig.write_image('./Pos_training/TS2Vec_head_model.pdf', format='pdf', width=1100, height=400)
fig.show()

In [202]:
fig_bar1 = px.bar(fragment_result.query("calibration == 'finetuning' & head_model == 'MLP'"), x='ssl_model', y='f1', color='scenario', 
             facet_row='calibration', facet_col='head_model', barmode='group',
             labels={'ssl_model': 'SSL Model', 'acc': 'Accuracy', 'scenario': 'Scenario'},
             )

fig_bar2 = px.bar(fragment_result.query("calibration == 'linearprobing' & head_model == 'MLP'"), x='ssl_model', y='f1', color='scenario', 
             facet_row='calibration', facet_col='head_model', barmode='group',
             labels={'ssl_model': 'SSL Model', 'acc': 'Accuracy', 'scenario': 'Scenario'},
             )

fig = make_subplots(rows=1, cols=2, subplot_titles=['Finetuning', 'Linear Probing'], horizontal_spacing=0.07,)

fig.add_traces(
    fig_bar1['data'],
    rows=1, cols=1
)

fig.add_traces(
    fig_bar2['data'],
    rows=1, cols=2,

)
fig.update_annotations(font_size=18)
fig.update_layout(showlegend=False)
fig.update_yaxes(title='F1-score', range=[0, 1])
fig.update_layout(height=420, width=1600, font_size=18, 
                  legend_title_text="",
                  margin=dict(l=40, r=40, b=30, t=80),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.07, xanchor="center", x=0.5)
)

In [161]:
fig = px.bar(fragment_result.query("head_model == 'MLP'"), x='ssl_model', y='f1', color='scenario', 
             facet_row='calibration', barmode='group',
             labels={'ssl_model': 'SSL Model', 'acc': 'Accuracy', 'scenario': 'Scenario', 'f1': 'F1-score'},
             category_orders={
                 'scenario': [
                    'one_domain', 'one_domain_extended', 
                    'one_domain_continued1', 'continued1_extended', 'one_domain_continued30',
                    'one_domain+Fragment',
                    'four_domains', 'four_domains_extended', 
                    'four_domains+Fragment',
                    ]
             },
             )

fig.update_yaxes(range=[0, 1])
fig.update_layout(height=600, width=900, font_size=16, 
                  legend_title_text="",
                  margin=dict(l=40, r=40, b=30, t=80),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.07, xanchor="center", x=0.5)
)

In [ ]:
fragment_supervised = pd.read_csv('./Supervised/Fragment_supervised.csv')
fragment_supervised.head()

,model,dataset,avg_loss,acc,acc_std,f1,f1_std,recall,recall_std,precision,precision_std
0,HinceptionTime,fragment,NaN,0.5706,0.0000,0.5468,0.0010,0.5706,0.0000,0.5407,0.0028
1,MLP,fragment,NaN,0.4601,0.0000,0.3840,0.0000,0.4601,0.0000,0.3380,0.0000
2,FCN,fragment,NaN,0.3337,0.0055,0.1859,0.0097,0.3337,0.0055,0.1295,0.0099


### IEEEPPG Results

In [12]:
ieeeppg_result = pd.read_csv('./Pos_training/ieeeppg_results.csv')
ieeeppg_result.head()

,ssl_model,head_model,calibration,scenario,dataset,avg_loss,mse,mse_std,r2_score,r2_score_std,rmse,rmse_std,mae,mae_std
0,TSTCC,MLP,linearprobing,one_domain,IEEEPPG,517.3157,0.0101,0.0043,22.7445,0.0498,18.3980,0.1768,529.4080,1.4312
1,TSTCC,MLP,linearprobing,four_domains,IEEEPPG,518.3404,0.0082,0.0030,22.7671,0.0346,18.4338,0.1247,529.0043,1.1918
2,TSTCC,MLP,finetuning,one_domain,IEEEPPG,535.3663,-0.0244,0.0264,23.1364,0.2993,19.1758,0.5177,543.2826,11.4943
3,TSTCC,MLP,finetuning,four_domains,IEEEPPG,604.8880,-0.1574,0.0264,24.5932,0.2804,21.0037,0.3759,591.9205,15.9592
4,TSTCC,FCN,linearprobing,one_domain,IEEEPPG,1190.1912,-1.2774,0.2310,34.4638,1.7447,30.7696,1.6269,1177.3956,119.0651


In [13]:
fig = px.bar(ieeeppg_result, x='ssl_model', y='rmse', color='scenario', 
             facet_row='calibration', facet_col='head_model', barmode='group')

fig.update_layout(height=640, width=1000, font_size=16, 
                  legend_title_text="",
                  margin=dict(l=40, r=40, b=30, t=80),
                  showlegend=True,
                  legend=dict(orientation="h", yanchor="bottom", y=1.07, xanchor="center", x=0.5)
)

In [137]:
import torch
import json
from models.TS2Vec import TS2Vec

In [135]:
ckp = torch.load("./Pos_training/TS2Vec_continued30/2025-05-07_10-01/MLP_finetuning_and_one_domain_best_ssl_model_weights.pth")

In [ ]:
ckp

In [141]:
config = json.load(open("../Results/Pre_Training/TS2Vec/2025-05-06_16-30/WESAD_continued30_TS2Vec_config.json"))
config

{'Model_Type': 'TS2Vec',
 'Training_mode': 'Pre_Training',
 'save_dir': 'Results/Pre_Training/TS2Vec/2025-05-06_16-30/checkpoints',
 'data_dir': 'Dataset/Benchmarks',
 'seed': 1234,
 'gpu': 0,
 'batch_size': 64,
 'optim_args': {'lr': 0.001, 'betas': [0.9, 0.999]},
 'model_args': {'in_dim': 1,
  'out_dim': 320,
  'hidden_dim': 64,
  'depth': 10,
  'kernel_size': 3,
  'dropout': 0.1,
  'mask_mode': 'binomial'},
 'epochs': 30,
 'problem': 'WESAD_no_ecg',
 'resume': True,
 'old-problem': 'WESAD_ECGonly',
 'old-config': 'Results/Pre_Training/TS2Vec/2025-04-15_16-04/WESAD_ECGonly_TS2Vec_config.json'}

In [144]:
encoder = TS2Vec(**config['model_args'])
encoder.load_state_dict(ckp)

In [145]:
encoder